In [2]:
"""
Molecular Feature Extraction for Reactivity Ratio Prediction

This code extracts physicochemical features from monomer SMILES structures. As detailed in Section 2.2 and 
Table 1 of our manuscript, we extract 15 key molecular descriptors that capture different 
aspects affecting polymerization kinetics, including vinyl group characteristics, electronic 
properties, steric factors, and other parameters that influence chain configuration.

"""

import warnings
warnings.filterwarnings('ignore')
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from PreprocessData import *
from Analyzer import *
from FingerPrint import *
from Descriptor import *
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

'Here, please put the directory of the processed dataset for extraction of feature from SMILES'

# Load the dataset containing monomer pairs and their experimentally determined reactivity ratios
file_path = address = r'/work/bavarian/hsafari2/Manuscript Code/Dataset.xlsx'

df = pd.read_excel(file_path)
print(f"Dataset loaded with {len(df)} rows")

# Drop the first two columns (monomer names) as we'll work with SMILES representations
df = df.drop(columns=['MonomerA', 'MonomerB'])

# Display the initial dataframe
print("\nInitial dataset after loading:")
display(df.head(5))
initial_row_count = len(df)


# Feature 1: Vinyl Group Position (Cyclic vs Linear)
# This feature determines whether the vinyl group is embedded within a ring structure
# or in a linear chain. As shown in Figure 3 of our manuscript, this structural 
# characteristic significantly impacts reactivity, with unringed vinyl groups
# showing higher reactivity (mean = 1.35) compared to ringed vinyl groups (mean = 0.67).
print("\nExtracting Feature 1: Vinyl Group Position...")
df['vinylIsInLoop_A'] = 0
df['vinylIsInLoop_B'] = 0
# Iterate through each row in the dataframe to calculate the feature for both SMILE A and SMILE B
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    mol_A = Chem.MolFromSmiles(str(smile_A))
    mol_B = Chem.MolFromSmiles(str(smile_B))
    
    # Check if the vinyl group is in a loop for both SMILE A and SMILE B
    if mol_A and VinylLoopIndicator.calculate(mol_A):
        df.at[index, 'vinylIsInLoop_A'] = 1
        
    if mol_B and VinylLoopIndicator.calculate(mol_B):
        df.at[index, 'vinylIsInLoop_B'] = 1


# Feature 2: Vinyl Carbon Charges
# The charge distribution around vinyl carbons is a critical determinant of radical
# stability during propagation. This electronic parameter directly influences the
# reactivity of the monomer by affecting the electron density at the reaction site.
print("Extracting Feature 2: Vinyl Carbon Charges...")
def calculate_vinyl_carbon_charge(smile):
    """Calculate the sum of carbon charges for vinyl groups in the given molecule."""
    mol = Chem.MolFromSmiles(smile)
    if mol:
        try:
            # Try to calculate the vinyl group carbon charges
            carbonCharges = PolarChargeCarbonVinyl.calculate(mol)
            return sum(carbonCharges)
        except:
            # If calculation fails for any reason, return 0
            return 0
    return 0  # Return 0 for molecules where calculation fails

# Initialize new columns for vinyl group charge feature for both SMILES A and SMILES B
df['VinylCarbonsCharge_A'] = 0
df['VinylCarbonsCharge_B'] = 0

# Iterate through each row in the dataframe to calculate the feature for both SMILE A and SMILE B
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']    
    # Calculate and store the vinyl carbon charge for both SMILES A and SMILES B
    df.at[index, 'VinylCarbonsCharge_A'] = calculate_vinyl_carbon_charge(smile_A)
    df.at[index, 'VinylCarbonsCharge_B'] = calculate_vinyl_carbon_charge(smile_B)


# Feature 3: Molecular Volume
# Molecular volume quantifies the spatial requirements of monomers, which affects
# steric interactions during propagation. Bulkier monomers may experience more
# steric hindrance, potentially reducing their reactivity and influencing the
# resulting polymer chain configuration.
print("Extracting Feature 3: Molecular Volume...")
def calculate_molecular_volume(smile):
    """Calculate molecular volume safely."""
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            return MolecularVolumeCalculator.calculate(mol)
    except:
        pass
    return 0  # Default value if calculation fails

# Initialize columns for molecular volume with default values
df['MV_A'] = 0
df['MV_B'] = 0

# Calculate molecular volume for each molecule
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    mv_A = calculate_molecular_volume(smile_A)
    mv_B = calculate_molecular_volume(smile_B)
    
    if mv_A > 0:  # Only update if calculation succeeded
        df.at[index, 'MV_A'] = mv_A
    if mv_B > 0:
        df.at[index, 'MV_B'] = mv_B


# Feature 4: Molecular Weight
# Molecular weight is a fundamental size parameter that influences diffusion and
# mobility during polymerization. It serves as a critical indicator of molecular
# size, directly affecting spatial arrangements and steric effects during propagation.
print("Extracting Feature 4: Molecular Weight...")
def calculate_molecular_weight(smile):
    """Calculate the molecular weight of the given molecule."""
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            return Descriptors.MolWt(mol)  # Using RDKit's built-in descriptor
    except:
        pass
    return 0  # Default value if calculation fails

# Initialize columns with default values
df['MW_A'] = 0
df['MW_B'] = 0

# Calculate molecular weight for each molecule
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    mw_A = calculate_molecular_weight(smile_A)
    mw_B = calculate_molecular_weight(smile_B)
    
    if mw_A > 0:  # Only update if calculation succeeded
        df.at[index, 'MW_A'] = mw_A
    if mw_B > 0:
        df.at[index, 'MW_B'] = mw_B


# Feature 5: Total Polar Surface Area (TPSA)
# TPSA quantifies the polar surface area of molecules, affecting intermolecular
# interactions during polymerization. This parameter influences solvation effects
# and potential hydrogen bonding interactions that can impact reactivity ratios.
print("Extracting Feature 5: Total Polar Surface Area...")
def calculate_tpsa(smile):
    """Calculate total polar surface area safely."""
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            return MolecularSurfaceAreaCalculator.calculate(mol)
    except:
        pass
    return 0  # Default value if calculation fails

# Initialize columns with default values
df['TPSA_A'] = 0
df['TPSA_B'] = 0

# Calculate TPSA for each molecule
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    tpsa_A = calculate_tpsa(smile_A)
    tpsa_B = calculate_tpsa(smile_B)
    
    if tpsa_A > 0:  # Only update if calculation succeeded
        df.at[index, 'TPSA_A'] = tpsa_A
    if tpsa_B > 0:
        df.at[index, 'TPSA_B'] = tpsa_B


# Feature 6: Number of Hydrogen Acceptors
# Hydrogen acceptors influence potential chain transfer sites and hydrogen bonding
# effects during polymerization. This parameter contributes to the reactivity and 
# interaction capabilities of monomers with growing polymer chains.
print("Extracting Feature 6: Hydrogen Acceptors...")
def calculate_h_acceptors(smile):
    """Calculate hydrogen acceptors safely."""
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            return Descriptors.NumHAcceptors(mol)
    except:
        pass
    return 0  # Default value if calculation fails

# Initialize columns with default values
df['NumHAcceptors_A'] = 0
df['NumHAcceptors_B'] = 0

# Calculate hydrogen acceptors for each molecule
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    acceptors_A = calculate_h_acceptors(smile_A)
    acceptors_B = calculate_h_acceptors(smile_B)
    
    df.at[index, 'NumHAcceptors_A'] = acceptors_A
    df.at[index, 'NumHAcceptors_B'] = acceptors_B


# Feature 7: Number of Hydrogen Donors
# Hydrogen donors, like acceptors, affect hydrogen bonding capabilities and potential
# chain transfer reactions. The balance of donors and acceptors influences the
# intermolecular interactions during polymerization.
print("Extracting Feature 7: Hydrogen Donors...")
def calculate_h_donors(smile):
    """Calculate hydrogen donors safely."""
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            return Descriptors.NumHDonors(mol)
    except:
        pass
    return 0  # Default value if calculation fails

# Initialize columns with default values
df['NumHDonors_A'] = 0
df['NumHDonors_B'] = 0

# Calculate hydrogen donors for each molecule
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    donors_A = calculate_h_donors(smile_A)
    donors_B = calculate_h_donors(smile_B)
    
    df.at[index, 'NumHDonors_A'] = donors_A
    df.at[index, 'NumHDonors_B'] = donors_B


# Feature 8: Molecular LogP (Lipophilicity)
# MolLogP quantifies the hydrophobic/hydrophilic balance of monomers, which affects
# solubility and phase behavior during polymerization. This parameter influences
# monomer distribution in heterogeneous systems and potentially their reactivity.
print("Extracting Feature 8: Molecular LogP...")
def calculate_logp(smile):
    """Calculate molecular logP safely."""
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            return Descriptors.MolLogP(mol)
    except:
        pass
    return 0  # Default value if calculation fails

# Initialize columns with default values
df['MolLogP_A'] = 0
df['MolLogP_B'] = 0

# Calculate logP for each molecule
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    logp_A = calculate_logp(smile_A)
    logp_B = calculate_logp(smile_B)
    
    df.at[index, 'MolLogP_A'] = logp_A
    df.at[index, 'MolLogP_B'] = logp_B


# Features 9-11: Carbon Hybridization States (sp, sp2, sp3)
# The hybridization states of carbon atoms determine electronic distribution and
# orbital overlap capabilities, directly affecting reactivity. sp² hybridization
# is particularly important for vinyl polymerization, while the balance of different
# hybridization states influences overall molecular reactivity.
print("Extracting Features 9-11: Carbon Hybridization States...")
def calculate_hybridization(smile):
    """Calculate carbon hybridization counts for a molecule."""
    # Initialize with zeros instead of None
    hybridization_counts = {'sp': 0, 'sp2': 0, 'sp3': 0}
    
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            # Process each atom in the molecule
            for atom in mol.GetAtoms():
                if atom.GetAtomicNum() == 6:  # Only count carbon atoms
                    # Get hybridization state
                    hyb = atom.GetHybridization()
                    
                    # Convert RDKit hybridization type to string representation
                    if hyb == Chem.rdchem.HybridizationType.SP:
                        hybridization_counts['sp'] += 1
                    elif hyb == Chem.rdchem.HybridizationType.SP2:
                        hybridization_counts['sp2'] += 1
                    elif hyb == Chem.rdchem.HybridizationType.SP3:
                        hybridization_counts['sp3'] += 1
    except:
        pass  # Keep the default zeros if there's an error
    
    return hybridization_counts

# Initialize hybridization columns with zeros
df['Hybridization_sp_A'] = 0
df['Hybridization_sp2_A'] = 0
df['Hybridization_sp3_A'] = 0
df['Hybridization_sp_B'] = 0
df['Hybridization_sp2_B'] = 0
df['Hybridization_sp3_B'] = 0

# Calculate hybridization for each molecule
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    hybridization_A = calculate_hybridization(smile_A)
    hybridization_B = calculate_hybridization(smile_B)
    
    # Always update with the calculated values (will be 0 if calculation failed)
    df.at[index, 'Hybridization_sp_A'] = hybridization_A['sp']
    df.at[index, 'Hybridization_sp2_A'] = hybridization_A['sp2']
    df.at[index, 'Hybridization_sp3_A'] = hybridization_A['sp3']
    
    df.at[index, 'Hybridization_sp_B'] = hybridization_B['sp']
    df.at[index, 'Hybridization_sp2_B'] = hybridization_B['sp2']
    df.at[index, 'Hybridization_sp3_B'] = hybridization_B['sp3']


# Feature 12: Chirality
# Chirality affects stereochemical effects on approach geometry during polymerization.
# The presence of chiral centers can influence the orientation of monomers during
# the propagation step, potentially affecting reactivity and chain configuration.
print("Extracting Feature 12: Chirality...")
def calculate_chirality(smile):
    """Calculate the number of chiral centers safely."""
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=True)
            return len(chiral_centers)
    except:
        pass
    return 0  # Default value if calculation fails

# Initialize columns with default values
df['Chirality_A'] = 0
df['Chirality_B'] = 0

# Calculate chirality for each molecule
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    chirality_A = calculate_chirality(smile_A)
    chirality_B = calculate_chirality(smile_B)
    
    df.at[index, 'Chirality_A'] = chirality_A
    df.at[index, 'Chirality_B'] = chirality_B


# Feature 13: Number of Connected Hydrogens
# The number of connected hydrogens relates to chain transfer potential and reactivity
# sites. Hydrogen atoms can participate in chain transfer reactions during
# polymerization, affecting propagation kinetics and chain termination.
print("Extracting Feature 13: Connected Hydrogens...")
def calculate_explicit_hydrogens(smile):
    """Calculate the number of hydrogen atoms safely."""
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            mol_with_h = Chem.AddHs(mol)  # Add explicit hydrogens
            num_hydrogens = sum([1 for atom in mol_with_h.GetAtoms() if atom.GetAtomicNum() == 1])
            return num_hydrogens
    except:
        pass
    return 0  # Default value if calculation fails

# Initialize columns with default values
df['NumConnectedHydrogens_A'] = 0
df['NumConnectedHydrogens_B'] = 0

# Calculate connected hydrogens for each molecule
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    hydrogens_A = calculate_explicit_hydrogens(smile_A)
    hydrogens_B = calculate_explicit_hydrogens(smile_B)
    
    df.at[index, 'NumConnectedHydrogens_A'] = hydrogens_A
    df.at[index, 'NumConnectedHydrogens_B'] = hydrogens_B


# Feature 14: Conjugated Bonds
# Conjugated bonds influence resonance stabilization and radical stability during
# polymerization. Extended conjugation can stabilize radical intermediates,
# potentially affecting reactivity and propagation kinetics.
print("Extracting Feature 14: Conjugated Bonds...")
def calculate_conjugated_bonds(smile):
    """Calculate the number of conjugated bonds safely."""
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            conjugated_bonds = sum([1 for bond in mol.GetBonds() if bond.GetIsConjugated()])
            return conjugated_bonds
    except:
        pass
    return 0  # Default value if calculation fails

# Initialize columns with default values
df['ConjugatedBonds_A'] = 0
df['ConjugatedBonds_B'] = 0

# Calculate conjugated bonds for each molecule
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    conj_bonds_A = calculate_conjugated_bonds(smile_A)
    conj_bonds_B = calculate_conjugated_bonds(smile_B)
    
    df.at[index, 'ConjugatedBonds_A'] = conj_bonds_A
    df.at[index, 'ConjugatedBonds_B'] = conj_bonds_B


# Feature 15: Stereochemistry
# Stereochemistry affects spatial arrangement and accessibility effects during
# polymerization. The three-dimensional arrangement of atoms can influence
# approach geometry and steric interactions during propagation.
print("Extracting Feature 15: Stereochemistry...")
def calculate_stereochemistry(smile):
    """Calculate stereochemistry (count of stereocenters) safely."""
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            stereochemistry = len(Chem.FindMolChiralCenters(mol, includeUnassigned=True))
            return stereochemistry
    except:
        pass
    return 0  # Default value if calculation fails

# Initialize columns with default values
df['Stereochemistry_A'] = 0
df['Stereochemistry_B'] = 0

# Calculate stereochemistry for each molecule
for index, row in df.iterrows():
    smile_A = row['SMILES_A']
    smile_B = row['SMILES_B']
    
    stereo_A = calculate_stereochemistry(smile_A)
    stereo_B = calculate_stereochemistry(smile_B)
    
    df.at[index, 'Stereochemistry_A'] = stereo_A
    df.at[index, 'Stereochemistry_B'] = stereo_B


# Arrange columns in a logical order matching the 15 features from Table 1
feature_columns = [
    'SMILES_A', 'SMILES_B',                                   # Molecular identifiers
    'vinylIsInLoop_A', 'vinylIsInLoop_B',                    # Feature 1: Vinyl position
    'VinylCarbonsCharge_A', 'VinylCarbonsCharge_B',          # Feature 2: Vinyl carbon charges
    'MV_A', 'MV_B',                                          # Feature 3: Molecular volume
    'MW_A', 'MW_B',                                          # Feature 4: Molecular weight
    'TPSA_A', 'TPSA_B',                                      # Feature 5: Total polar surface area
    'NumHAcceptors_A', 'NumHAcceptors_B',                    # Feature 6: H-bond acceptors
    'NumHDonors_A', 'NumHDonors_B',                          # Feature 7: H-bond donors
    'MolLogP_A', 'MolLogP_B',                                # Feature 8: Lipophilicity
    'Hybridization_sp_A', 'Hybridization_sp_B',              # Feature 9: sp hybridization
    'Hybridization_sp2_A', 'Hybridization_sp2_B',            # Feature 10: sp² hybridization
    'Hybridization_sp3_A', 'Hybridization_sp3_B',            # Feature 11: sp³ hybridization
    'Chirality_A', 'Chirality_B',                            # Feature 12: Chirality
    'NumConnectedHydrogens_A', 'NumConnectedHydrogens_B',    # Feature 13: Connected hydrogens
    'ConjugatedBonds_A', 'ConjugatedBonds_B',                # Feature 14: Conjugated bonds
    'Stereochemistry_A', 'Stereochemistry_B',                # Feature 15: Stereochemistry
    'r1', 'r2'                                               # Target variables
]

# Select only columns that exist in the dataframe
existing_columns = [col for col in feature_columns if col in df.columns]
df = df[existing_columns]

# Print diagnostic information
print("\nData processing summary:")
print(f"Initial number of rows: {initial_row_count}")
print(f"Final number of rows: {len(df)}")
print(f"Features extracted: {len(existing_columns) - 2 - 2}")  # Subtract SMILES and r1,r2

# Verify all feature columns are present
missing_columns = [col for col in feature_columns if col not in df.columns]
if missing_columns:
    print(f"Warning: Some expected columns are missing: {missing_columns}")
else:
    print("All 15 feature types successfully extracted for both monomers")

# Display the final dataframe with all extracted features
print("\nFinal dataset with all 15 molecular features from Table 1 of manuscript:")
display(df.head(10))

# Save the final dataset with all extracted features for subsequent clustering analysis
output_path = r'/work/bavarian/hsafari2/Manuscript Code/AddedFeatures.xlsx'
df.to_excel(output_path, index=False)

print(f"\nThe dataset has been successfully saved to: {output_path}")
print("This dataset is now ready for the clustering analysis described in Section 2.3 of our manuscript")

Dataset loaded with 2304 rows

Initial dataset after loading:


,SMILES_A,SMILES_B,r1,r2
0,CC(=C)C(=O)O,C=C(Cl)Cl,3.368,0.154
1,C=C(Cl)Cl,CC(=C)C(=O)O,0.154,3.368
2,CC(=C)C(=O)O,CCOC(=O)C(=C)C,0.570,0.710
3,CCOC(=O)C(=C)C,CC(=C)C(=O)O,0.710,0.570
4,CC(=C)C(=O)O,CC(C)COC(=O)C(=C)C,2.010,0.470



Extracting Feature 1: Vinyl Group Position...
Extracting Feature 2: Vinyl Carbon Charges...
Extracting Feature 3: Molecular Volume...
Extracting Feature 4: Molecular Weight...
Extracting Feature 5: Total Polar Surface Area...
Extracting Feature 6: Hydrogen Acceptors...
Extracting Feature 7: Hydrogen Donors...
Extracting Feature 8: Molecular LogP...
Extracting Features 9-11: Carbon Hybridization States...
Extracting Feature 12: Chirality...
Extracting Feature 13: Connected Hydrogens...
Extracting Feature 14: Conjugated Bonds...
Extracting Feature 15: Stereochemistry...

Data processing summary:
Initial number of rows: 2304
Final number of rows: 2304
Features extracted: 30
All 15 feature types successfully extracted for both monomers

Final dataset with all 15 molecular features from Table 1 of manuscript:


,SMILES_A,SMILES_B,vinylIsInLoop_A,vinylIsInLoop_B,VinylCarbonsCharge_A,VinylCarbonsCharge_B,MV_A,MV_B,MW_A,MW_B,...,Chirality_A,Chirality_B,NumConnectedHydrogens_A,NumConnectedHydrogens_B,ConjugatedBonds_A,ConjugatedBonds_B,Stereochemistry_A,Stereochemistry_B,r1,r2
0,CC(=C)C(=O)O,C=C(Cl)Cl,0,0,-0.062175,0.030563,84.360,70.160,86.090,96.944,...,0,0,6,2,4,0,0,0,3.368,0.154
1,C=C(Cl)Cl,CC(=C)C(=O)O,0,0,0.030563,-0.062175,70.160,84.360,96.944,86.090,...,0,0,2,6,0,4,0,0,0.154,3.368
2,CC(=C)C(=O)O,CCOC(=O)C(=C)C,0,0,-0.062175,-0.061991,84.344,118.208,86.090,114.144,...,0,0,6,10,4,4,0,0,0.570,0.710
3,CCOC(=O)C(=C)C,CC(=C)C(=O)O,0,0,-0.061991,-0.062175,117.992,84.240,114.144,86.090,...,0,0,10,6,4,4,0,0,0.710,0.570
4,CC(=C)C(=O)O,CC(C)COC(=O)C(=C)C,0,0,-0.062175,-0.061991,84.360,151.856,86.090,142.198,...,0,0,6,14,4,4,0,0,2.010,0.470
5,CC(C)COC(=O)C(=C)C,CC(=C)C(=O)O,0,0,-0.061991,-0.062175,151.720,84.344,142.198,86.090,...,0,0,14,6,4,4,0,0,0.470,2.010
6,CC(=C)C(=O)O,CC(=C)C(=O)OCC1CO1,0,0,-0.062175,-0.061989,84.304,136.568,86.090,142.154,...,0,1,6,10,4,4,0,1,0.980,1.200
7,CC(=C)C(=O)OCC1CO1,CC(=C)C(=O)O,0,0,-0.061989,-0.062175,136.864,84.304,142.154,86.090,...,1,0,10,6,4,4,1,0,1.200,0.980
8,CC(=C)C(=O)O,CC(=O)OC=C,0,0,-0.062175,0.023785,84.360,84.752,86.090,86.090,...,0,0,6,6,4,4,0,0,0.200,0.010
9,CC(=O)OC=C,CC(=C)C(=O)O,0,0,0.023785,-0.062175,84.632,84.304,86.090,86.090,...,0,0,6,6,4,4,0,0,0.010,0.200



The dataset has been successfully saved to: /work/bavarian/hsafari2/Manuscript Code/AddedFeatures.xlsx
This dataset is now ready for the clustering analysis described in Section 2.3 of our manuscript


In [3]:
"""
Monomer Extraction and Reactivity Analysis

This script transforms the copolymer dataset into a unique monomer dataset and analyzes reactivity 
patterns. As described in Section 2.1 of our manuscript, we process experimental reactivity ratio 
data to understand the variability and distribution of monomer behaviors across different 
copolymerization experiments.

The code extracts unique monomers from the copolymer dataset, collects all reactivity ratios 
associated with each monomer, and calculates statistical metrics (mean, standard deviation, etc.) 
to quantify their reactivity patterns. 
"""

import pandas as pd
import numpy as np

'Here, please use the  Monomer Physicochemical Features excel file that is shared with you.'

# Load the dataset containing molecular features extracted in the previous step
df = pd.read_excel(r'/work/bavarian/hsafari2/Manuscript Code/AddedFeatures.xlsx')

# Round r1 and r2 to three decimal places to reduce noise in experimental measurements
# This helps address the inherent variability in experimental data as mentioned in Section 2.1
df['r1'] = df['r1'].round(3)
df['r2'] = df['r2'].round(3)

# Create a dictionary to store unique monomers and their properties
# This restructures data from copolymer pairs to individual monomers
monomer_dict = {}

# Iterate through each row in the dataset to extract individual monomers
for index, row in df.iterrows():
    # Extract Monomer A properties from the copolymer dataset
    smiles_A = row['SMILES_A']
    properties_A = {
        'SMILES': smiles_A,
        'MW': row['MW_A'],
        'vinylIsInLoop': row['vinylIsInLoop_A'],
        'VinylCarbonsCharge': row['VinylCarbonsCharge_A'],
        'MV': row['MV_A'],
        'TPSA': row['TPSA_A'],
        'NumHAcceptors': row['NumHAcceptors_A'],
        'NumHDonors': row['NumHDonors_A'],
        'MolLogP': row['MolLogP_A'],
        'Hybridization_sp': row['Hybridization_sp_A'],
        'Hybridization_sp2': row['Hybridization_sp2_A'],
        'Hybridization_sp3': row['Hybridization_sp3_A'],
        'Chirality': row['Chirality_A'],
        'NumConnectedHydrogens': row['NumConnectedHydrogens_A'],
        'ConjugatedBonds': row['ConjugatedBonds_A'],
        'Stereochemistry': row['Stereochemistry_A'],
        'ReactivityRatios': []  # List to collect all reactivity ratios for this monomer
    }
    
    # Extract Monomer B properties from the copolymer dataset
    smiles_B = row['SMILES_B']
    properties_B = {
        'SMILES': smiles_B,
        'MW': row['MW_B'],
        'vinylIsInLoop': row['vinylIsInLoop_B'],
        'VinylCarbonsCharge': row['VinylCarbonsCharge_B'],
        'MV': row['MV_B'],
        'TPSA': row['TPSA_B'],
        'NumHAcceptors': row['NumHAcceptors_B'],
        'NumHDonors': row['NumHDonors_B'],
        'MolLogP': row['MolLogP_B'],
        'Hybridization_sp': row['Hybridization_sp_B'],
        'Hybridization_sp2': row['Hybridization_sp2_B'],
        'Hybridization_sp3': row['Hybridization_sp3_B'],
        'Chirality': row['Chirality_B'],
        'NumConnectedHydrogens': row['NumConnectedHydrogens_B'],
        'ConjugatedBonds': row['ConjugatedBonds_B'],
        'Stereochemistry': row['Stereochemistry_B'],
        'ReactivityRatios': []  # List to collect all reactivity ratios for this monomer
    }
    
    # Store Monomer A if not already in dictionary, or update its reactivity ratios
    if smiles_A not in monomer_dict:
        monomer_dict[smiles_A] = properties_A
    # Append the reactivity ratio r1 to monomer A's list
    monomer_dict[smiles_A]['ReactivityRatios'].append(row['r1'])
    
    # Store Monomer B if not already in dictionary, or update its reactivity ratios
    if smiles_B not in monomer_dict:
        monomer_dict[smiles_B] = properties_B
    # Append the reactivity ratio r2 to monomer B's list
    monomer_dict[smiles_B]['ReactivityRatios'].append(row['r2'])

# Convert the dictionary of monomers to a DataFrame for easier analysis
unique_monomer_df = pd.DataFrame.from_dict(monomer_dict, orient='index')

# Reset the index to have a clean DataFrame
unique_monomer_df.reset_index(drop=True, inplace=True)

# Calculate statistical metrics for each monomer's reactivity behavior
# These metrics help assess consistency and reliability as described in Equation 3 of the manuscript
unique_monomer_df['NumReactivityRatios'] = unique_monomer_df['ReactivityRatios'].apply(len)
unique_monomer_df['MeanReactivityRatio'] = unique_monomer_df['ReactivityRatios'].apply(np.mean)
unique_monomer_df['StdReactivityRatio'] = unique_monomer_df['ReactivityRatios'].apply(np.std)
unique_monomer_df['MinReactivityRatio'] = unique_monomer_df['ReactivityRatios'].apply(min)
unique_monomer_df['MaxReactivityRatio'] = unique_monomer_df['ReactivityRatios'].apply(max)

# Sort monomers by their data richness (number of experimental observations)
# This helps identify which monomers have been most extensively studied
unique_monomer_df.sort_values(by='NumReactivityRatios', ascending=False, inplace=True)
unique_monomer_df.reset_index(drop=True, inplace=True)

# Display summary statistics for the dataset
print("\nDataset Summary:")
print(f"Total number of unique monomers: {len(unique_monomer_df)}")
print("\nDistribution of number of reactivity ratios per monomer:")
print(unique_monomer_df['NumReactivityRatios'].value_counts().sort_index())

# Print statistics about reactivity ratio distribution
# This helps understand the overall reactivity landscape across the dataset
print("\nReactivity Ratio Statistics:")
print(f"Average number of reactivity ratios per monomer: {unique_monomer_df['NumReactivityRatios'].mean():.2f}")
print(f"Maximum number of reactivity ratios for a monomer: {unique_monomer_df['NumReactivityRatios'].max()}")
print(f"Minimum number of reactivity ratios for a monomer: {unique_monomer_df['NumReactivityRatios'].min()}")

display(unique_monomer_df)

# Save the processed dataset for further analysis in the clustering stage
unique_monomer_df.to_excel(r'/work/bavarian/hsafari2/Manuscript Code/featuresInvestigationOnReactivityratio.xlsx', index=False)


Dataset Summary:
Total number of unique monomers: 628

Distribution of number of reactivity ratios per monomer:
NumReactivityRatios
2      345
4      127
6       58
8       27
10      18
12       8
14      10
16       6
18       4
20       4
22       2
26       1
28       3
30       1
34       1
36       1
38       2
44       1
46       1
60       1
66       2
96       1
110      1
248      1
414      1
702      1
Name: count, dtype: int64

Reactivity Ratio Statistics:
Average number of reactivity ratios per monomer: 7.34
Maximum number of reactivity ratios for a monomer: 702
Minimum number of reactivity ratios for a monomer: 2


,SMILES,MW,vinylIsInLoop,VinylCarbonsCharge,MV,TPSA,NumHAcceptors,NumHDonors,MolLogP,Hybridization_sp,...,Chirality,NumConnectedHydrogens,ConjugatedBonds,Stereochemistry,ReactivityRatios,NumReactivityRatios,MeanReactivityRatio,StdReactivityRatio,MinReactivityRatio,MaxReactivityRatio
0,C=CC1=CC=CC=C1,104.152,0,-0.160678,111.720,0.00,0,0,2.32960,0,...,0,8,8,0,"[5.2, 5.2, 0.25, 0.25, 0.193, 0.193, 0.52, 0.5...",702,1.023068,1.495560,0.020,8.920
1,CC(=C)C(=O)OC,100.117,0,-0.062003,101.008,26.30,2,0,0.73550,0,...,0,8,4,0,"[8.99, 8.99, 0.1, 0.1, 0.418, 0.418, 6.02, 6.0...",414,1.357246,1.495633,0.080,10.000
2,C=CC#N,53.064,0,-0.080055,57.776,23.79,1,0,0.69598,1,...,0,3,3,0,"[0.59, 0.59, 0.138, 0.138, 0.122, 0.122, 0.217...",248,1.175411,1.715992,0.010,9.090
3,COC(=O)C=C,86.090,0,-0.075811,84.944,26.30,2,0,0.34540,0,...,0,6,4,0,"[0.4, 0.4, 0.14, 0.14, 0.02, 0.02, 0.18, 0.18,...",110,0.772182,1.088337,0.013,5.800
4,CC(=O)OC=C,86.090,0,0.023785,84.752,26.30,2,0,0.69300,0,...,0,6,4,0,"[0.01, 0.01, 0.15, 0.15, 0.021, 0.021, 1.55, 1...",96,0.843667,1.445844,0.010,8.270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,CC(C)(C)[Si](C)(C)OC1=CC=C(C=C1)C=C,234.415,0,-0.160673,251.544,9.23,1,0,4.71360,0,...,0,22,9,0,"[0.863, 0.863]",2,0.863000,0.000000,0.863,0.863
624,CCCCN(CCCC)C(=O)C=C,183.295,0,-0.097702,205.848,20.31,1,0,2.60120,0,...,0,21,4,0,"[0.294, 0.294]",2,0.294000,0.000000,0.294,0.294
625,C=CC1=NC=NC=C1,106.128,0,-0.140442,102.776,25.78,2,0,1.11960,0,...,0,6,8,0,"[2.38, 2.38]",2,2.380000,0.000000,2.380,2.380
626,CCOC(=O)C=CC(=O)OC,158.153,0,0.060877,146.752,52.60,4,0,0.27870,0,...,0,10,7,0,"[0.02, 0.02]",2,0.020000,0.000000,0.020,0.020


In [4]:

import pandas as pd
import numpy as np
# --- paths ---
src_path = r'/work/bavarian/hsafari2/Manuscript Code/featuresInvestigationOnReactivityratio.xlsx'
out_path = r'/work/bavarian/hsafari2/Manuscript Code/UniqueMonomersProperties.xlsx'

# --- read source ---
df = pd.read_excel(src_path)

# --- keep only these columns, in this order ---
keep_cols = [
    'SMILES','MW','vinylIsInLoop','VinylCarbonsCharge','MV','TPSA','NumHAcceptors',
    'NumHDonors','MolLogP','Hybridization_sp','Hybridization_sp2','Hybridization_sp3',
    'Chirality','NumConnectedHydrogens','ConjugatedBonds','Stereochemistry'
]

missing = [c for c in keep_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns in source: {missing}")

df = df.loc[:, keep_cols].copy()

# --- clean: map common text booleans, coerce to numeric, fill empties with 0 ---
# (we do this for all columns EXCEPT SMILES, which is text)
bool_map = {'True':1,'False':0,'true':1,'false':0,'Yes':1,'No':0,'yes':1,'no':0, True:1, False:0}
for c in df.columns:
    if c == 'SMILES':
        continue
    if df[c].dtype == 'O':
        df[c] = df[c].replace(bool_map)

num_cols = [c for c in df.columns if c != 'SMILES']
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')
df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan).fillna(0)

# (optional) if you want to DROP rows with empty SMILES, uncomment:
# df = df[df['SMILES'].astype(str).str.strip() != '']

# --- save cleaned subset ---
df.to_excel(out_path, index=False)
print(f"Cleaned file saved to: {out_path}")

# --- continue using the cleaned dataframe in the rest of your code ---
# (this ensures downstream PCA/t-SNE/UMAP uses the cleaned columns only)


Cleaned file saved to: /work/bavarian/hsafari2/Manuscript Code/UniqueMonomersProperties.xlsx
